In [19]:
import pandas as pd
from datetime import datetime
import pandas as np

Información

- AccidentesPlantas: De momento no se va a usar. Hay grado de movilidad que se podria añadir.
- Balande hidroeléctrico: De momento no se va a usar.
- ConstantesVitales: Se extraer la tensión y la temperatura, pero en la mayoria de casos no hay datos. ----
- CurasVendajes: De momento no se va a usar.
- DatosIdentificativosAdministrativos:  !!!!!! IMPORTANTE ----
- DiagnósticosEnfermería: Mejor usar DiagnosticosEnfermeriaCodificados
- DiagnosticosEnfermeriaCodificados: De momento no se usa. 'gprobenf' y 'dprobenf' puede que sea importante. -----
- DiagnosticosMedicosCodificados: De momento no se usa. 'gdiagein', 'gcodidia',	'dnombdia'
- DietasViasAlimentacion: De momento  no se van a usar.
- InterconsultasSinDocumento: De momento no se va a usar.
- IntervencionesRealizadas: De momento no se va a usar.
- ManifestadoPor: De momento no se va a usar.
- Medidas: De momento no se va a usar.No info.
- OrdenesMedicas: De momento no se va a usar.
- PlanTipicoAsignado: De momento no se va a usar.No info.
- Prestaciones: De momento no se va a usar.
- ProcedimientosMedicosCodificados: De momento no se va a usar.
- RelacionadoCon: De momento no se va a usar.
- UlcerasPresion:De momento no se va a usar.



-PrescripcionFarmacos: POR HACER
- ValoracionesAntecedentesPersonales: POR HACER
- ValoracionesEnfermeria: POR HACER
- ValoracionesMedicas: POR HACER

- Valoraciones medicas: De momento no se van a usar. No contienen información que podamos utilizar. En 'ovallcon' se podría obtener algo de informacion como 'izquierda o derecha'



Carga de archivos

In [20]:
df = pd.read_csv('datos/DatosIdentificativosAdministrativos.csv')
df_constantes = pd.read_csv('datos/ConstantesVitales.csv')
df_glucemias = pd.read_csv('datos/GlucemiasGlucosurias.csv')
df_accidentes = pd.read_csv('datos/AccidentesPlanta.csv')
df_dietas = pd.read_csv('datos/DietasViasAlimentacion.csv')
df_antecedentes = pd.read_csv('datos/ValoracionesAntecedentesPersonales.csv')
df_valoracion_enfermeria = pd.read_csv('datos/ValoracionesEnfermeria.csv')




Valoraciones médicas

In [21]:
# Eliminar las columnas que no tienen información relevante
df = df.drop(columns=['odiagalt', 'gdiasec1', 'odiasec1', 'gdiasec2', 'odiasec2','itraslad', 'dmotalta','dsitalta','dnomserv','gserulti','gdiagini','gdiaging','dmotingr','gmotingr','gnumadme','ganoadme','dpoblaci','dnomprov','odiaging'	,'odiagini' ])

In [22]:

#Nos quedamos solo con las fracturas de cadera. Empiezan todas por S72.
df = df[df['gdiagalt'].str.startswith('S72', na=False)]

# Eliminar filas duplicadas basadas en la columna 'gidenpac', conservando solo la primera aparición. Pasamos de 1078 a 1047 filas
df = df.drop_duplicates(subset='gidenpac', keep='first')

# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Listas de códigos para "DERECHO" y "IZQUIERDO"
derecho_codes = ["001", "011", "051", "061", "091", "101", "111", "131", "141", "21", "301", "341", "391", "411", "421", "451", "491", "8X1", "91X"]
izquierdo_codes = ["002", "012", "042", "092", "102", "112", "115", "142", "22", "302", "425", "435", "452", "455", "492", "92X"]

# Categorizar la columna
def categorize_side(value):
    value = str(value)  # Convertir el valor a string por seguridad
    if any(code in value for code in derecho_codes):
        return 0
    elif any(code in value for code in izquierdo_codes):
        return 1
    else:
        return None  # Marcar para eliminar

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)

# Eliminar las filas donde 'ds_izq_der' es None
df = df.dropna(subset=['ds_izq_der'])   # Solo perdemos 1
# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------


# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime
df['fllegada'] = pd.to_datetime(df['fllegada'])
df['faltplan'] = pd.to_datetime(df['faltplan'])
df['finterve'] = pd.to_datetime(df['finterve'])

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days

# Calcular los días de preoperatorio (finterve - fllegada)
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days

# Calcular los días de postoperatorio (faltplan - finterve)
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
# ------------------------------------------------------------------------------------------

# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

/tmp/ipykernel_2748/500377688.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})
/tmp/ipykernel_2748/500377688.py:89: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})


In [23]:
print(len(df))

1046


In [24]:
df

,gidenpac,itipsexo,fnacipac,gcodipos,gprovinc,gpoblaci,fllegada,itipingr,ireingre,finterve,...,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada
16,MRMR192402260,0,1924-02-26,24006.0,24.0,242127.0,2024-09-23 19:35:05,I,0,2024-09-25,...,S72.142A,1.0,1,100,4,1.0,3.0,0,1,9
32,GRMN192606210,0,1926-06-21,24328.0,24.0,241396.0,2019-03-14 21:31:42,I,0,2019-03-22,...,S72.012A,1.0,1,98,18,7.0,11.0,1,4,3
111,BLGR192807170,0,1928-07-17,24006.0,24.0,240896.0,2020-02-07 15:29:51,I,0,2020-02-10,...,S72.012A,1.0,1,96,7,2.0,4.0,1,5,2
165,LFLL192212310,0,1922-12-31,24240.0,24.0,241572.0,2021-06-24 18:00:04,I,0,2021-06-28,...,S72.141A,0.0,1,101,7,3.0,4.0,1,4,6
237,DRCL192405020,0,1924-05-02,24600.0,24.0,241149.0,2019-04-07 01:19:40,I,0,2019-04-16,...,S72.012A,1.0,2,100,17,8.0,8.0,1,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42957,BZMR193509440,1,1935-09-04,24010.0,24.0,240896.0,2020-05-19 01:35:37,I,0,2020-05-22,...,S72.012A,1.0,2,89,8,2.0,5.0,1,2,5
42958,LVGR193509470,1,1935-09-07,24001.0,24.0,240896.0,2019-04-09 15:10:26,I,0,2019-04-10,...,S72.012A,1.0,1,89,7,0.0,7.0,1,2,4
42962,RDDM193509620,1,1935-09-22,24358.0,24.0,240358.0,2019-05-02 19:42:10,I,0,2019-05-03,...,S72.141A,0.0,1,89,14,0.0,14.0,1,4,5
43035,CBCS194203680,1,1942-03-28,24240.0,24.0,241572.0,2024-04-08 21:35:44,I,0,2024-04-12,...,S72.141A,0.0,1,82,15,3.0,12.0,1,1,4


Accidentes


In [25]:
# Filtrar df_accidentes para seleccionar solo las columnas relevantes
df_accidentes = df_accidentes[['gidenpac', 'ifamilia']]

# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'
df = pd.merge(df, df_accidentes, on='gidenpac', how='left')

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'
df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})

/tmp/ipykernel_2748/2138728134.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})


Dietas


Constantes vitales

In [26]:
#La mayoria están vacios. Seria mejor eliminarlos ?

# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo
df_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')
df_constantes = df_constantes.groupby('gidenpac').first().reset_index()

# Realizar el merge con df, añadiendo las constantes vitales
df = pd.merge(df, df_constantes, on='gidenpac', how='left')

Antecedentes


In [27]:
# Seleccionar solo las columnas necesarias de 'df_antecedentes' y crear una copia
df_ant = df_antecedentes[['gidenpac', 'dconclin', 'vbivalor']].copy()

# Crear columnas en 'df_ant' basadas en el contenido de 'dconclin'
df_ant['ds_HTA'] = df_ant["dconclin"].str.contains("HTA", na=False).astype(int)
df_ant['ds_alergia_medicamentosa'] = df_ant["dconclin"].str.contains("Alergia medicamentosa", na=False).astype(int)
df_ant['ds_alergia_alimenticia'] = df_ant["dconclin"].str.contains("Alergia alimenticia", na=False).astype(int)
df_ant['ds_diabetes'] = df_ant["dconclin"].str.contains("Diabetes Mellitus", na=False).astype(int)
df_ant['ds_otras_alergias'] = df_ant["dconclin"].str.contains("Otras alergias", na=False).astype(int)

# Actualizar valores en columnas de antecedentes usando 'vbivalor' si no es nulo
columns_to_update = ['ds_HTA', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias']
for col in columns_to_update:
    df_ant[col] = df_ant.apply(lambda x: x['vbivalor'] if pd.notnull(x['vbivalor']) and x[col] == 1 else x[col], axis=1)

# Reemplazar valores específicos ('N' -> 0, 'S' -> 1, 'A' -> 1) en todo el DataFrame
df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)

# Eliminar las columnas 'dconclin' y 'vbivalor' si ya no son necesarias
df_ant.drop(columns=['dconclin', 'vbivalor'], inplace=True)

# Eliminar duplicados en el DataFrame (si aún no se ha hecho)
df_ant = df_ant.drop_duplicates()

# Agrupar por 'gidenpac' y usar 'max' para que cualquier 1 en una columna mantenga el valor como 1
df_ant = df_ant.groupby('gidenpac', as_index=False).max()

# Realizar un merge entre df y df_ant sin sufijos
df = pd.merge(df, df_ant, on='gidenpac', how='left')

# Asignar valores de df_ant solo si son 1
for col in columns_to_update:
    df[col] = df.apply(lambda x: x[col] if pd.notnull(x[col]) and x[col] == 1 else np.nan if pd.isna(x[col]) else x[col], axis=1)


/tmp/ipykernel_2748/2302255109.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)


In [28]:
df_ant

,gidenpac,ds_HTA,ds_alergia_medicamentosa,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias
0,BBMR195406141,1,0,0,1,0
1,BBRV196104300,0,0,0,0,0
2,BBXX193502600,0,0,0,0,0
3,BCCB200202020,0,0,0,0,0
4,BCCN193804700,0,0,0,1,0
...,...,...,...,...,...,...
33000,ZTLN196202550,0,0,0,1,0
33001,ZYML195212020,0,0,0,0,0
33002,ZZBD198908430,0,0,0,0,0
33003,ZZXX197406420,0,0,0,0,0


In [29]:
df

,gidenpac,itipsexo,fnacipac,gcodipos,gprovinc,gpoblaci,fllegada,itipingr,ireingre,finterve,...,ifamilia,ntensmin,ntensmax,ntempera,nsatuoxi,ds_HTA,ds_alergia_medicamentosa,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias
0,MRMR192402260,0,1924-02-26,24006.0,24.0,242127.0,2024-09-23 19:35:05,I,0,2024-09-25,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,GRMN192606210,0,1926-06-21,24328.0,24.0,241396.0,2019-03-14 21:31:42,I,0,2019-03-22,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2,BLGR192807170,0,1928-07-17,24006.0,24.0,240896.0,2020-02-07 15:29:51,I,0,2020-02-10,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
3,LFLL192212310,0,1922-12-31,24240.0,24.0,241572.0,2021-06-24 18:00:04,I,0,2021-06-28,...,NaN,NaN,NaN,NaN,NaN,0,0,1,1,0
4,DRCL192405020,0,1924-05-02,24600.0,24.0,241149.0,2019-04-07 01:19:40,I,0,2019-04-16,...,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,BZMR193509440,1,1935-09-04,24010.0,24.0,240896.0,2020-05-19 01:35:37,I,0,2020-05-22,...,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0
1053,LVGR193509470,1,1935-09-07,24001.0,24.0,240896.0,2019-04-09 15:10:26,I,0,2019-04-10,...,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0
1054,RDDM193509620,1,1935-09-22,24358.0,24.0,240358.0,2019-05-02 19:42:10,I,0,2019-05-03,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1055,CBCS194203680,1,1942-03-28,24240.0,24.0,241572.0,2024-04-08 21:35:44,I,0,2024-04-12,...,0.0,NaN,NaN,NaN,NaN,0,1,0,1,0


Valoracion Enfermeria

In [30]:

# Seleccionar solo las columnas necesarias de 'df_valoracion_enfermeria'
df_valoracion_enfermeria = df_valoracion_enfermeria[['gidenpac', 'dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']].copy()


# Definir las expresiones regulares de los valores que queremos conservar en 'dconclin'
patterns = [
    "_Movilidad",
    "Movilidad",
    "G Vive Solo",
    "G Lugar de Residencia",
    "G Procedencia",
    "G Destino Alta",
    "Resultado Indice de Barthel",
    "Resultado Indice de Barthel al Alta",
    "Resultado Escala de Braden",
    "Resultado Escala Riesgo Caidas"
]

# Crear una expresión regular unida para filtrar las filas deseadas
regex_pattern = "|".join(patterns)

# Filtrar el DataFrame para conservar solo las filas que coinciden con el patrón
df_valoracion_enfermeria = df_valoracion_enfermeria[df_valoracion_enfermeria['dconclin'].str.contains(regex_pattern, na=False)]


'''
# Crear columnas basadas en 'dconclin' usando expresiones regulares
df_valoracion_enfermeria['_movilidad'] = df_valoracion_enfermeria['dconclin'].str.contains("_Movilidad", na=False).astype(int)
df_valoracion_enfermeria['movilidad'] = df_valoracion_enfermeria['dconclin'].str.contains("Movilidad", na=False).astype(int)
df_valoracion_enfermeria['vive_solo'] = df_valoracion_enfermeria['dconclin'].str.contains("G Vive Solo", na=False).astype(int)
df_valoracion_enfermeria['lugar_residencia'] = df_valoracion_enfermeria['dconclin'].str.contains("G Lugar de Residencia", na=False).astype(int)
df_valoracion_enfermeria['lugar_procedencia'] = df_valoracion_enfermeria['dconclin'].str.contains("G Procedencia", na=False).astype(int)
df_valoracion_enfermeria['destino_alta'] = df_valoracion_enfermeria['dconclin'].str.contains("G Destino Alta", na=False).astype(int)
df_valoracion_enfermeria['Barthel'] = df_valoracion_enfermeria['dconclin'].str.contains("Resultado Indice de Barthel", na=False).astype(int)
df_valoracion_enfermeria['barthel_alta'] = df_valoracion_enfermeria['dconclin'].str.contains("Resultado Indice de Barthel al Alta", na=False).astype(int)
df_valoracion_enfermeria['braden'] = df_valoracion_enfermeria['dconclin'].str.contains("Resultado Escala de Braden", na=False).astype(int)
df_valoracion_enfermeria['riesgo_caida'] = df_valoracion_enfermeria['dconclin'].str.contains("Resultado Escala Riesgo Caidas", na=False).astype(int)

df_valoracion_enfermeria['vbivalor'] = df_valoracion_enfermeria['vbivalor'].replace({'N': 0, 'S': 1})


# Reemplazar NaN con 0 en todas las columnas numéricas para que no afecten el cálculo de max
df_valoracion_enfermeria.fillna(0, inplace=True)

# Convertir las columnas 'ncodtabu', 'nvalncon', y 'vbivalor' a numérico
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'vbivalor']].apply(pd.to_numeric, errors='coerce')

# Opcional: Si deseas que las columnas sean de tipo int y llenar NaNs con 0
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'vbivalor']].fillna(0).astype(int)

# Eliminar la columna 'ovallcon' del DataFrame
df_valoracion_enfermeria = df_valoracion_enfermeria.drop(columns=['ovallcon']) 



# Agrupar por 'gidenpac' y aplicar max() para mantener el valor más alto en cada columna
df_valoracion_enfermeria = df_valoracion_enfermeria.groupby('gidenpac', as_index=False).max()



# Separar las columnas no numéricas
non_numeric_cols = ['dconclin', 'vbivalor']
df_non_numeric = df_valoracion_enfermeria[non_numeric_cols + ['gidenpac']]


# Eliminar las columnas no numéricas del DataFrame original
df_numeric = df_valoracion_enfermeria.drop(columns=non_numeric_cols)

# Aplicar max() solo a las columnas numéricas, agrupando por 'gidenpac'
df_numeric = df_numeric.groupby('gidenpac').max().reset_index()


# Unir de nuevo las columnas no numéricas al DataFrame resultante
df_valoracion_enfermeria = pd.merge(df_numeric, df_non_numeric.drop_duplicates(subset=['gidenpac']), on='gidenpac', how='left')

# Crear columnas adicionales para almacenar valores específicos donde la condición es 1
def assign_columns(df, col_name, source_cols):
    for suffix in source_cols:
        df[f"{col_name}_{suffix}"] = np.where(df[col_name] == 1, df[suffix], np.nan)

# Aplicar la función de asignación de columnas
source_columns = ['ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon']
assign_columns(df_valoracion_enfermeria, '_movilidad', source_columns)
assign_columns(df_valoracion_enfermeria, 'movilidad', source_columns)
assign_columns(df_valoracion_enfermeria, 'lugar_residencia', source_columns)
assign_columns(df_valoracion_enfermeria, 'lugar_procedencia', source_columns)
assign_columns(df_valoracion_enfermeria, 'destino_alta', source_columns)

# Asignar valores de 'ovallcon' para escalas y convertir a números si es necesario
scales = ['Barthel', 'barthel_alta', 'braden', 'riesgo_caida']
for scale in scales:
    df_valoracion_enfermeria[f"{scale}_ovallcon"] = np.where(df_valoracion_enfermeria[scale] == 1, df_valoracion_enfermeria['ovallcon'], np.nan)
    df_valoracion_enfermeria[f"{scale}_ovallcon"] = pd.to_numeric(df_valoracion_enfermeria[f"{scale}_ovallcon"], errors='coerce')

# Reemplazar valores de 'vive_solo' con vbivalor y mapear N y S
df_valoracion_enfermeria["vive_solo"] = np.where(df_valoracion_enfermeria["vive_solo"] == 1, df_valoracion_enfermeria["vbivalor"], np.nan)
df_valoracion_enfermeria['vive_solo'].replace({'N': 0, 'S': 1}, inplace=True)

# Eliminar columnas que ya no son necesarias
cols_to_drop = ['dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']
cols_to_drop += ['Barthel', 'barthel_alta', 'braden', 'riesgo_caida']
df_valoracion_enfermeria.drop(columns=cols_to_drop, inplace=True)

# Mostrar el resultado final
print(df_valoracion_enfermeria.head())
'''

'\n# Crear columnas basadas en \'dconclin\' usando expresiones regulares\ndf_valoracion_enfermeria[\'_movilidad\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("_Movilidad", na=False).astype(int)\ndf_valoracion_enfermeria[\'movilidad\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("Movilidad", na=False).astype(int)\ndf_valoracion_enfermeria[\'vive_solo\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("G Vive Solo", na=False).astype(int)\ndf_valoracion_enfermeria[\'lugar_residencia\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("G Lugar de Residencia", na=False).astype(int)\ndf_valoracion_enfermeria[\'lugar_procedencia\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("G Procedencia", na=False).astype(int)\ndf_valoracion_enfermeria[\'destino_alta\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("G Destino Alta", na=False).astype(int)\ndf_valoracion_enfermeria[\'Barthel\'] = df_valoracion_enfermeria[\'dconclin\'].str.contains("Resultado 

In [31]:
df_valoracion_enfermeria

,gidenpac,dconclin,ncodtabu,nsecuval,nvalncon,ovallcon,vbivalor
0,PLFR192406120,Movilidad,2.0,1,NaN,NaN,NaN
3,PLFR192406120,Resultado Indice de Barthel,NaN,1,NaN,5,NaN
21,MRFR192902410,Resultado Escala de Braden,NaN,1,NaN,16,NaN
41,MRSN193402540,Movilidad,2.0,1,NaN,NaN,NaN
45,RDLV193007530,Resultado Escala de Braden,NaN,1,NaN,17,NaN
...,...,...,...,...,...,...,...
1630,FDFD192912290,Movilidad,2.0,1,NaN,NaN,NaN
1632,FDFD192912290,Resultado Indice de Barthel,NaN,1,NaN,30,NaN
1633,FDFD192912290,_Movilidad,NaN,1,2.0,NaN,NaN
1639,LZCS192812620,G Lugar de Residencia,1.0,1,NaN,NaN,NaN


In [32]:
df_valoracion_enfermeria.to_csv('DATOS3.csv', index=False)


Glucemias

In [33]:
# Pocos datos

# Crear un conjunto con los valores de gidenpac en df_glucemias
glucemias_pacientes = set(df_glucemias['gidenpac'].dropna())

# Crear la columna 'ds_diabetes' solo en las casillas vacías
df['ds_diabetes'] = df.apply(lambda row: 1 if pd.isna(row['ds_diabetes']) and row['gidenpac'] in glucemias_pacientes 
                             else (0 if pd.isna(row['ds_diabetes']) else row['ds_diabetes']), axis=1)



In [34]:
unique_count = df['gidenpac'].nunique()
print("Número de valores únicos en la columna 'gidenpac':", unique_count)


Número de valores únicos en la columna 'gidenpac': 1046


In [35]:
df.to_csv('DATOS2.csv', index=False)


In [36]:
df

,gidenpac,itipsexo,fnacipac,gcodipos,gprovinc,gpoblaci,fllegada,itipingr,ireingre,finterve,...,ifamilia,ntensmin,ntensmax,ntempera,nsatuoxi,ds_HTA,ds_alergia_medicamentosa,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias
0,MRMR192402260,0,1924-02-26,24006.0,24.0,242127.0,2024-09-23 19:35:05,I,0,2024-09-25,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,GRMN192606210,0,1926-06-21,24328.0,24.0,241396.0,2019-03-14 21:31:42,I,0,2019-03-22,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2,BLGR192807170,0,1928-07-17,24006.0,24.0,240896.0,2020-02-07 15:29:51,I,0,2020-02-10,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
3,LFLL192212310,0,1922-12-31,24240.0,24.0,241572.0,2021-06-24 18:00:04,I,0,2021-06-28,...,NaN,NaN,NaN,NaN,NaN,0,0,1,1,0
4,DRCL192405020,0,1924-05-02,24600.0,24.0,241149.0,2019-04-07 01:19:40,I,0,2019-04-16,...,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,BZMR193509440,1,1935-09-04,24010.0,24.0,240896.0,2020-05-19 01:35:37,I,0,2020-05-22,...,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0
1053,LVGR193509470,1,1935-09-07,24001.0,24.0,240896.0,2019-04-09 15:10:26,I,0,2019-04-10,...,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0
1054,RDDM193509620,1,1935-09-22,24358.0,24.0,240358.0,2019-05-02 19:42:10,I,0,2019-05-03,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1055,CBCS194203680,1,1942-03-28,24240.0,24.0,241572.0,2024-04-08 21:35:44,I,0,2024-04-12,...,0.0,NaN,NaN,NaN,NaN,0,1,0,1,0
